# Problem Set 1 (PS1): A Deeper Dive into Malkiel's Theorem 4
Malkiel proposed five theorems that govern the price of fixed-income debt securities, e.g., Treasury Bills, Notes, and Bonds, as a function of the duration, yield, and coupon (interest) rate  values:

> [Malkiel, B. G. (1962). Expectations, Bond Prices, and the Term Structure of Interest Rates. The Quarterly Journal of Economics, 76(2), 197–218. https://doi.org/10.2307/1880816](https://www.jstor.org/stable/1880816)

In class, we showed (by simulation) that we could observe the essential stipulation of `Theorem 4`: Price movements resulting from equal absolute increases and decreases in yield are asymmetric; i.e., decreasing yields raise bond prices more than the same increase in yields lowers prices. However, during the discussion of the simulation results, we ran up against an interesting question: 
> __Question__: Would you expect the price asymmetry to increase or decrease with the duration of the note or bond?

### Tasks
This problem set's objective is to explore the price asymmetry question we introduced in the lecture by simulating price changes in different instruments resulting from changes in $\bar{r}$.
* __Prerequisite__: Load the Treasury auction dataset. We load some data and set some stuff that we use below. There is nothing to do in the __prerequisite__ section, but check it out if you are curious.
* __Task 1__: Compute prices for a Bill, Note, and Bond to check our implementation. We've selected some Treasury securities from the dataset, but we must build models and compute prices (and other data). In this task, you will create (and populate) models of a Bill, a Note, and a Bond and compare the computed price with the price observed at auction.
* __Task 2__: In class, we showed that notes (and, by extension, bonds) show price change asymmetry. However, we have not explored whether Bills also have this property. In this task, you'll use the treasury bill model we created above and explore the price asymmetry question by simulation.
* __Task 3__: Finally, in class, we showed that coupon securities exhibited price asymmetry, i.e., the absolute percentage change in the price was larger for decreases in $\bar{r}\downarrow$ compared to the response when $\bar{r}\uparrow$ increases. However, are longer-maturity instruments more sensitive to this effect, i.e., do they exhibit a larger asymmetry? Let's do some calculations to explore this question.

## Setup
Set up the computational environment using [the `Include.jl` file](Include.jl). The [`Include.jl` file](Include.jl) loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [ ]:
include("Include.jl");

## Prerequisite: Load the Treasury auction dataset
We'll explore `T-bill,` `T-note,` and `T-bond` prices from United States Treasury auctions between March 2020 and September 2024 downloaded as a `CSV` file using the [Auction query functionality of TreasuryDirect.gov](https://www.treasurydirect.gov/auctions/auction-query/). We load the `CSV` dataset using the local [`MyTreasuryBillsNotesAndBondsDataSet()` function](src/Files.jl), which returns the auction data as [the `dataset::DataFrame`](https://dataframes.juliadata.org/stable/) variable.

In [ ]:
dataset = MyTreasuryBillsNotesAndBondsDataSet()

To help below, let's use [the `filter!(...)` method exported by the DataFrames.jl package](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) to collect only the data for the bills; we save this to the `bills_dataset::DataFrame` variable.

In [ ]:
bills_dataset = filter(Symbol("Security Type") => x-> x == "Bill", dataset)

Similarly, we use [the `filter!(...)` method exported by the DataFrames.jl package](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) to collect only the data for the notes and bonds; we save this to the `notes_bonds_dataset::DataFrame` variable.

In [ ]:
notes_bonds_dataset = filter(Symbol("Security Type") => x-> (x == "Note" || x == "Bond"), dataset)

Finally, we build an [instance of the `DiscreteCompoundingModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.DiscreteCompoundingModel) and store this discount model in the `discount_model` variable:

In [ ]:
discount_model = DiscreteCompoundingModel();

## Task 1: Compute prices for a Bill, Note, and Bond to check our implementation
To verify our code installation (and implementation), the teaching team has selected a random bill, note, and bond from the dataset. In this task, you will construct the corresponding treasury model instance, compute the price for each of these securities, and compare the calculated and observed prices. 

### Check: Compute the price of a treasury bill
Let's compute the price of a `26-Week` T-bill with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04865.` The observed price for this security is $V_{B}$ = `97.6315 USD`. Assume two compounding periods per year.
* Create an instance of [the `MyUSTreasuryZeroCouponBondModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryZeroCouponBondModel) using a [custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), assuming the data above. Pass the data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), and use the short-cut syntax to compute the price. Save the updated model in the `zero_coupon_model` variable. 

In [ ]:
#On the Fixed Income Treasury Secuirty of Prof. Varners textbook, I learned that Par is face value of the bond, n is the two compounding periods, T is a 26 week old 
#bong, rbar is the discount rate, and VB is the observed price. Given the example in this text book, I constructed the following code where we populate 
#MyUSTreasuryCouponSecurityModel with the instances provided.
zero_coupon_model = build(MyUSTreasuryZeroCouponBondModel, (
    n = 2, par = 100, 
        T = "26-Week" |> securityterm,
        rate = 0.04865,
)) |> discount_model;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [ ]:
observed_bill_price = 97.6315;
@assert isapprox(observed_bill_price, zero_coupon_model.price, rtol=1e-4)

### Check: Compute the price of a treasury note
Let's compute the price of a `7-Year` T-Note with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04162` and coupon rate $\bar{c}$ = `0.04125.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.
* Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple})., assuming the data above. Pass in the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `note_coupon_model` variable. 

In [ ]:
#Similarly from the previous code, we calculate the T-Note through new values of Lambda (which was n before), and now include T as 7 years and introduce the concept
# of a coupon, which is the annual intrest paid on a bond. 
note_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
    λ = 2, par = 100.0,
        
    T = "7-Year" |> securityterm,
    rate = 0.04162,
    coupon = 0.04125,
)) |> discount_model;
#all of this gets pushed into the discount_model

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [ ]:
observed_note_price = 99.7773;
@assert isapprox(observed_note_price, note_coupon_model.price, rtol=1e-4)
#from lecture we can copy the @assert isapprox. First save in the bond price, then save that into the coupon model

### Check: Compute the price of a treasury bond
Let's compute the price of a `30-Year` T-Bond with par value $V_{P}$ = `100 USD` with a value of $\bar{r}$ = `0.04314` and coupon rate $\bar{c}$ = `0.0425.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.
* Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple})., assuming the data above. Pass in the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `bond_coupon_model` variable. 

In [ ]:
bond_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
    λ = 2, par = 100.0,
        
    T = "30-Year" |> securityterm,
    rate = 0.04314,
    coupon = 0.0425,
    
)) |> discount_model;
#similarly to the past two examples, the only thing we change is the coupon, rate, and T-bond values. The notation and everything else stays consistent

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 
* If the computed and observed price is different, i.e., they have relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError).

In [ ]:
observed_bond_price = 98.9288;
@assert isapprox(observed_bond_price, bond_coupon_model.price, rtol=1e-4)

## Task 2: Do bills show the price asymmetry property? 
In class, we showed that notes (and, by extension, bonds) show price change asymmetry. However, we have not explored whether Bills also have this property. In this task, you'll use the treasury bill model that we created above and explore the price asymmetry question by simulation.

To simulate the asymmetry of changes in price following changes in the yield (discount rate), all other values held constant generate a new rate of the form $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value; if $\beta<1$ the perturbed interest rate is _less than_ the nominal rate, if $\beta=1$ the perturbed interest rate is _equals_ the nominal rate, and if $\beta>1$ the perturbed interest rate is _greater than_ the nominal rate. Let's use the `zero_coupon_model` instance from above for this experiment.

* First, specify the number of perturbation values in the `number_of_samples_task_2` variable; for this simulation, let `number_of_samples_task_2 = 7` (odd, so we capture the nominal case as the center data point).
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, compute the perturbation array (stored in the `β::Array{Float64,1}` variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array

In [ ]:
number_of_samples_task_2  = 7;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁, stop = β₂, length = number_of_samples_task_2) |> collect;

`Finish`: Your job is to complete the implementation of the `Theorem 4` simulation for Bills, which we started below, and analyze the simulation results. We save the simulation results in the `simulation_results_task_2_array::Array{Float64,2}` array and display the results in a table using the `pretty_table(...)` function exported from the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl)
*  `Hint`: check out [the Julia `deepcopy(...)` function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy). When you complete your implementation and run the notebook, you should see a table similar (numbers different, but similar structure) to the in-class example. Do you see that?

In [ ]:
simulation_results_task_2_array = let
    
    # initialize -
    VB = observed_bill_price;
    simulation_results_task_2_array = Array{Float64,2}(undef, number_of_samples_task_2, 3);

    # main loop
    for i ∈ eachindex(β)
        β_value = β[i]
       
        ### TODO: Build a deep copy of the zero-coupon model and update its rate. Alternatively, build a new model using the zcm parameters, and update the rate
        model = deepcopy(zero_coupon_model);
        model.rate = β_value*zero_coupon_model.rate
    
        #deep copy creates a copy of the object so that it becomes independent of the model.
        #from lecture, we know that the new rate is the perturbed value times original discount rate
        
        # compute: use short-cut syntax and compute the price
        perturbed_price = model |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_task_2_array
        simulation_results_task_2_array[i,1] = β_value; # col 1: perturbation value
        simulation_results_task_2_array[i,2] = 100*((model.rate - zero_coupon_model.rate)/(zero_coupon_model.rate));    # col 2: percentage change in r̄
        simulation_results_task_2_array[i,3] = 100*((model.price - VB)/(VB)); # col 3: percentage change in the price of the Bill
    end
    simulation_results_task_2_array
end;

In [ ]:
pretty_table(simulation_results_task_2_array, header=["β","Δr̄ (%)","ΔPrice (%)"] , tf=tf_simple)

#### Does Theorem 4 hold for T-bills?
Based on your analysis of the table above, update [the `Bool` value](https://docs.julialang.org/en/v1/base/numbers/#Core.Bool) below (either `true` or `false`):

In [ ]:
does_theorem_4_hold_for_bills = true; # update this value {true | false}

__Why?__: The answer for this is true. From the chart created, we can see the effects of how the perturbed values change with the discount rate. Therefore, we can also say that T-bills are very sensitive to changes in the discount rate, therefore also 
showing price change asymmetry, just like bonds and notes. Therefore, theorem 4 also holds for bills.

## Task 3: Is price price asymmetry a function of maturity?
Finally, in class, we showed that coupon securities exhibited price asymmetry, i.e., the absolute percentage change in the price was larger for decreases in $\bar{r}\downarrow$ compared to the response when $\bar{r}\uparrow$ increases. However, are longer-maturity instruments more sensitive to this effect, i.e., do they exhibit a larger asymmetry? Let's do some calculations to explore this question.

To start, create a hypothetical `T = 20-year` with the same parameters as the `30-year` bond we created above and compute its price. All parameters should be the same __expect__ the duration. Save the populated model instance in the `short_duration_bond` variable:

In [ ]:
#I know that Prof. Varner usually uses the systax where you push something into another thing, but soemtimes it confuses me a bit. Therefore,
#I set the T property of short_duration_bond to a 20 year secutiy term. This code is meant tomodel a 20 year term
short_duration_bond = deepcopy(bond_coupon_model)
short_duration_bond.T = securityterm("20-Year")
discount_model(short_duration_bond)

`Finish`: Next, finish implementing the code block below. This code computes the percentage change in the short- and long-maturity bond price and stores the simulation data in the `simulation_results_task_3_array::Array{Float64,2}` array. See the `TODO` block.

In [ ]:
simulation_results_task_3_array = let
    
    # initialize -
    simulation_results_task_3_array = Array{Float64,2}(undef, number_of_samples_task_2, 4);
    VB_short = short_duration_bond.price # original
    VB_long = bond_coupon_model.price
    VBn_short=short_duration_bond.rate
    VBn_long=bond_coupon_model.rate

    # main loop -
    for i ∈ eachindex(β)
        β_value = β[i]
    
        model_short = deepcopy(short_duration_bond); # create a copy of the short-duration model
        model_long = deepcopy(bond_coupon_model); # create a copy of the original bond model (long duration)
        
        ### TODO: update the rate for the model_short, and model_long instances -
        model_short.rate = β_value*VBn_short
        model_long.rate = β_value*VBn_long
        ### END 
        
        # compute: use short-cut syntax and compute the price of the short- and long-duration bonds
        perturbed_price_short = model_short |> discount_model |> x-> x.price
        perturbed_price_long = model_long |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_task_3_array
        simulation_results_task_3_array[i,1] = β_value; # col 1: store the perturbation value
        simulation_results_task_3_array[i,2] = 100*((model_short.rate - short_duration_bond.rate)/(short_duration_bond.rate));    # col 2: percentage change in r̄
        simulation_results_task_3_array[i,3] = 100*((model_short.price - VB_short)/(VB_short)); # col 3: percentage change in the price of the short bond
        simulation_results_task_3_array[i,4] = 100*((model_long.price - VB_long)/(VB_long)); # col 4: percentage change in the price of the long bond
    end
    
    simulation_results_task_3_array
end;

In [ ]:
pretty_table(simulation_results_task_3_array, header=["β","Δr̄ (%)", "ΔPrice-short (%)", "ΔPrice-long (%)"] , tf=tf_simple)

#### Does price asymmetry change with duration?
Based on your analysis of the table above, update [the `Bool` value](https://docs.julialang.org/en/v1/base/numbers/#Core.Bool) below (either `true` or `false`):

In [ ]:
is_price_asymmetry_func_maturity = true; # update this value {true | false}

__Why?__: the percentage in bond prices increases more as the discount rate decreases because the long duration of the bond become way more senstive to changes. The reason for this is because there is a higher chnace of things occuring within the given time frame. Therefore, yes there is a larger asymmetry because cash flow is way more distant and can't account for things in the future as well (more risk!)

## Testing
In the code block below, we compare your answers to the teaching team and give you feedback on which items are the same (which presumably means they are correct) and which are different. `Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

In [ ]:
let

    # load teaching team solution data -
    saved_solution_dict = joinpath(_PATH_TO_SOLN,"PS1-TeachingTeam-Solution-CHEME-5660-Fall-2024.jld2") |> load;

    @testset verbose = true "CHEME 5660 problem set 1 test suite" begin

        @testset "zero-coupon model tests" begin
            zero_model_soln = saved_solution_dict["zero_coupon_model"];

            # check the data on the zero model -
            @test zero_model_soln.T == zero_coupon_model.T
            @test zero_model_soln.n == zero_coupon_model.n
            @test zero_model_soln.par == zero_coupon_model.par
            @test zero_model_soln.rate == zero_coupon_model.rate
            @test isapprox(zero_model_soln.price, zero_coupon_model.price, atol=1e-4)
            @test zero_model_soln.cashflow == zero_coupon_model.cashflow
            @test zero_model_soln.discount == zero_coupon_model.discount
        end

        @testset "note model tests" begin
            note_model_soln = saved_solution_dict["note_coupon_model"];

            # check the data on the note -
            @test note_model_soln.T == note_coupon_model.T
            @test note_model_soln.λ == note_coupon_model.λ
            @test note_model_soln.par == note_coupon_model.par
            @test note_model_soln.rate == note_coupon_model.rate
            @test note_model_soln.coupon == note_coupon_model.coupon
            @test isapprox(note_model_soln.price, note_coupon_model.price, atol=1e-4)
            @test note_model_soln.cashflow == note_coupon_model.cashflow
            @test note_model_soln.discount == note_coupon_model.discount
        end

        @testset "bond model tests" begin
            bond_model_soln = saved_solution_dict["bond_coupon_model"];

            # check the data on the note -
            @test bond_model_soln.T == bond_coupon_model.T
            @test bond_model_soln.λ == bond_coupon_model.λ
            @test bond_model_soln.par == bond_coupon_model.par
            @test bond_model_soln.rate == bond_coupon_model.rate
            @test bond_model_soln.coupon == bond_coupon_model.coupon
            @test isapprox(bond_model_soln.price, bond_coupon_model.price, atol=1e-4)
            @test bond_model_soln.cashflow == bond_coupon_model.cashflow
            @test bond_model_soln.discount == bond_coupon_model.discount
        end

        @testset "Task 2 tests" begin
            simulation_results_task_2_soln = saved_solution_dict["simulation_results_task_2_array"];
            task_2_flag = saved_solution_dict["does_theorem_4_hold_for_bills"];
            
            @test round.(simulation_results_task_2_soln, digits=4) == round.(simulation_results_task_2_array, digits=4)
            @test does_theorem_4_hold_for_bills == task_2_flag
        end

        @testset "Task 3 tests" begin
            simulation_results_task_3_soln = saved_solution_dict["simulation_results_task_3_array"];
            task_3_flag = saved_solution_dict["is_price_asymmetry_func_maturity"];
            
            @test round.(simulation_results_task_3_soln, digits=4) == round.(simulation_results_task_3_array, digits=4)
            @test is_price_asymmetry_func_maturity == task_3_flag
        end
    end
end;

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.